last modified: 2023.11.26 23:30

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Advanced research for Tree model

[default features]

1. `count`: total number of packets
2. `incoming_count`: total number of incoming packets
3. `incoming_rate`: incoming_count / count
4. `outgoing_count`: total number of outgoing packets
5. `outgoing_rate`: outgoing_count / count

+ `initial_{n}_{feature}`: number of feature among first n packets

[sequential features]
1. timestamps
2. burst pattern
2. cumulative pattern

In [2]:
import pickle
import numpy as np
import pandas as pd
from itertools import chain

In [3]:
"""
파일의 폴더 위치 (절대경로 및 상대경로 무관)

ex)
/root
  ㄴ data
    ㄴ mon_standard.pkl
    ㄴ unmon_standard10.pkl

=> BASE_PATH = "/root/data/"
"""
# BASE_PATH = "./data/"
BASE_PATH = "/content/drive/MyDrive/Colab Notebooks/23-02 Machine Learning/project/"

In [4]:
# data 선택
FT_UNMON = True
FT_MON = True

# label type - False 인 경우 monitored(1), unmonitored(-1)
FT_MULTI_LABEL = True

### Feature engineering

- 각 feature를 넣어서 accuracy, confusion matrix를 뽑아 보는 시도가 필요함
- 여러 사람이 다양한 시도를 하고 있기 때문에 실험 하기 전에 노션에 시도할 실험에 대해 기록 후 카카오톡으로 남겨주세요.
- notion: https://www.notion.so/Experiment-e8275f06bf0f44b8b7b20172c374dd06

`FT_CATEGORICAL_SIZE`: categorical features 대해 집계할 point 배열.
- incoming count/rate, outgoing count/rate에 대한 중간 집계 시점을 결정함.
- 주어진 data 상에서 배열의 최소 길이는 50이지만, 만약 50보다 큰 값을 넣는 경우 모자란 길이만큼 뒤에 0으로 패딩처리 됨.
- 즉, [1, 2, 3, 4, 5, 6, ..., 50]을 다 넣는 경우 50번째 패킷까지 누적값을 계속 집계함.

`FT_SEQ_SIZE`: sequential features을 몇번째 packet까지 반영할지 결정함.
- True로 설정된 sequential features에 대해 분리함.
    - `FT_TIMESTAMPS`
    - `FT_DIRECTION`
    - `FT_DIRECTION`
    - `FT_BURST_DIR`
    - `FT_BURST_SIZE`
    - `FT_CUMULATIVE_DIR`
    - `FT_CUMULATIVE_SIZE`
- 즉, FT_TIMESTAMPS=True, FT_SEQ_SIZE=10인 경우, timestamp0 ~ timestamp9 column이 생김
- feature마다 다른 size로 넣는 것도 가능함: `seqeunce_features_size` 직접 수정
- timestamp의 경우, 첫번째는 다 0으로 동일하므로 생략함.

In [5]:
PACKET_SIZE = 512

#categorical
FT_CATEGORICAL = True
FT_CATEGORICAL_SIZE = range(10, 60, 10) # 10부터 10씩 늘려감, 60보다 작을 때까지. -> (10, 20, 30, 40, 50)
FT_SEQ_SIZE = 50

# feature 선택
# continuous
FT_TIMESTAMPS = True

FT_DIRECTION = True # 방향만 표기 [1. -1, 1, 1, ...]
FT_PACKET_SIZE = False # packet의 size로 표기 [512, -512, 512, 512, ...]


# busrt와 cumulative의 경우, Packet size를 반영하여 계산할 수도 있지만 Tor에서는 모든 패킷이 512 단위이므로, 그냥 scale을 줄여서 사용할 수도 있을거 같습니다.
# (저희에게 주어진 데이터의 경우 모두 512이므로 그냥 방향만으로 계산한다고 생각해도 무방)
FT_BURST_DIR = True # [1, -1, 1, 2, ...]
FT_BURST_SIZE = False # burst_dir에 packet size를 반영한 것 (burst_dir * 512)
FT_CUMULATIVE_DIR = True # [1, 0, 1, 2, ...]
FT_CUMULATIVE_SIZE = False # cumulative dir에 packet size를 반영한 것 (cumulative_dir * 512)

In [6]:
seqeunce_features_size = {
    'timestamps' :FT_SEQ_SIZE,
    'direction' :FT_SEQ_SIZE,
    'packet_size' :FT_SEQ_SIZE,
    'burst_dir' :FT_SEQ_SIZE,
    'burst_size' :FT_SEQ_SIZE,
    'cumulative_dir' :FT_SEQ_SIZE,
    'cumulative_size' :FT_SEQ_SIZE
    }

In [7]:
def load_pickle_file(file_name):
    # Load the pickle file
    print("Loading datafile...")
    with open(BASE_PATH + file_name, 'rb') as fi: # Path to mon_standard.pkl in Colab
        data = pickle.load(fi)

    print("Done.")
    return data

In [8]:
SITE_CNT = 95
URL_PER_SITE = 10

def parse_dataset(mon_data, unmon_data):
    print("parsing dataset...")
    dataset = []
    label = []

    if mon_data:
        if FT_MULTI_LABEL:
            label = [i for i in range(95) for _ in range(200)]
        else:
            label = [1] * 95 * 200

        for i in range(SITE_CNT):
            temp = []
            for j in range(URL_PER_SITE):
                temp.append(mon_data[i*URL_PER_SITE + j])
            dataset.append(list(chain.from_iterable(temp)))

    if unmon_data:
        label.extend([-1]*10000)
        dataset.append(unmon_data)


    dataset = list(chain.from_iterable(dataset))
    print("Done.")
    return dataset, label

In [9]:
if FT_MON:
    mon_data = load_pickle_file("mon_standard.pkl")
else:
    mon_data = None
if FT_UNMON:
    unmon_data = load_pickle_file("unmon_standard10.pkl")
else:
    unmon_data = None

dataset, label = parse_dataset(mon_data, unmon_data)

Loading datafile...
Done.
Loading datafile...
Done.
parsing dataset...
Done.


메모리 확보

In [10]:
del mon_data
del unmon_data

In [11]:
def pad_right(arr, size):
    if len(arr) >= size:
        return arr[:size]
    return list(np.pad(arr, (0, max(0, size - len(arr))), mode='constant'))


def calculate_burst_pattern(arr):
    if not arr:
        return []
    prev = arr[0]
    result = []

    for dir in arr[1:]:
        if (prev * dir) > 0:
            prev += dir
        else:
            result.append(prev)
            prev = dir

    result.append(prev)
    return result


def unfold_array(arr, size, col_name):
    return pd.DataFrame(map(lambda x:pad_right(x, size), arr), columns=[f"{col_name}_{i}" for i in range(size)])

def get_categorical_features(direction_arr, prefix):
    df = pd.DataFrame()

    length = list(map(len, direction_arr))

    df[f"{prefix}_incoming_count"] = list(map(lambda x:x.count(-1), direction_arr))
    df[f"{prefix}_incoming_rate"] = np.array(df[f"{prefix}_incoming_count"]) / length
    df[f"{prefix}_outgoing_count"] = length - np.array(df[f"{prefix}_incoming_count"])
    df[f"{prefix}_outgoing_rate"] = np.array(df[f"{prefix}_outgoing_count"]) / length
    return df

def extract_features(direction_arr, timestamps_arr):
    df = pd.DataFrame()

    columns = []

    if FT_TIMESTAMPS:
        columns.append(unfold_array(timestamps_arr, seqeunce_features_size['timestamps'], 'timestamp'))
        columns[0] = columns[0].drop(columns=['timestamp_0'])

    if FT_DIRECTION:
        columns.append(unfold_array(direction_arr, seqeunce_features_size['direction'], 'direction'))


    if FT_PACKET_SIZE:
        packet_size = list(map(lambda x: np.array(x) * PACKET_SIZE, direction_arr))
        columns.append(unfold_array(packet_size, seqeunce_features_size['packet_size'], 'packet_size'))

    if FT_BURST_DIR or FT_BURST_SIZE:
        burst = list(map(calculate_burst_pattern, direction_arr))
        if FT_BURST_DIR:
            columns.append(unfold_array(burst, seqeunce_features_size['burst_dir'], 'burst_dir'))
        if FT_BURST_SIZE:
            burst = list(map(lambda x: np.array(x) * PACKET_SIZE, burst))
            columns.append(unfold_array(burst, seqeunce_features_size['burst_size'], 'burst_size'))

    if FT_CUMULATIVE_DIR or FT_CUMULATIVE_SIZE:
        cumulative = list(map(lambda x: np.cumsum(x), direction_arr))
        if FT_CUMULATIVE_DIR:
            columns.append(unfold_array(cumulative, seqeunce_features_size['cumulative_dir'], 'cumulative_dir'))
        if FT_CUMULATIVE_SIZE:
            cumulative = list(map(lambda x:x * PACKET_SIZE, cumulative))
            columns.append(unfold_array(cumulative, seqeunce_features_size['cumulative_size'], 'cumulative_size'))

    if FT_CATEGORICAL:
        df["count"] = list(map(lambda x:len(x), direction_arr))
        columns.append(get_categorical_features(direction_arr, ''))

        for i in FT_CATEGORICAL_SIZE:
            columns.append(get_categorical_features(list(map(lambda x:pad_right(x, i), direction_arr)), f"initial{i}_"))

    return pd.concat([df, *columns], axis=1)

In [12]:
timestamps = []
direction = []

for data in dataset:
    timestamps.append(abs(np.array(data)))
    direction.append(list(map(lambda x: 1 if x > 0 else -1, data)))

del dataset

df = extract_features(direction, timestamps)
df['label'] = label

메모리 확보

In [13]:
del label
del timestamps
del direction

In [14]:
from sklearn.model_selection import train_test_split

class Dataset:
    x = None
    y = None

    def __init__(self, x, y, random_state=0, test_size=0.2):
        self.x = x
        self.y = y
        self.x_train, self.x_test, self.y_train, self.y_test = train_test_split(x, y, test_size=test_size, random_state=random_state)

## 사용 예시

In [15]:
open_multi_dataset = Dataset(df.drop(columns=['label']), df['label'])
open_binary_dataset = Dataset(df.drop(columns=['label']), df['label'].map(lambda x:1 if x>= 0 else -1))
closed_multi_dataset = Dataset(df[df['label'] >= 0].drop(columns=['label']), df[df['label'] >= 0]['label'])

In [16]:
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.model_selection import GridSearchCV

In [17]:
param_grid = { 'n_estimators':[10, 50, 100],
              'criterion': ['gini', 'entropy']}

tree_grid = GridSearchCV(RandomForestClassifier(), param_grid, refit = True, verbose = 3)
tree_grid.fit(open_multi_dataset.x_train, open_multi_dataset.y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV 1/5] END ...criterion=gini, n_estimators=10;, score=0.917 total time=   2.4s
[CV 2/5] END ...criterion=gini, n_estimators=10;, score=0.902 total time=   2.5s
[CV 3/5] END ...criterion=gini, n_estimators=10;, score=0.908 total time=   1.5s
[CV 4/5] END ...criterion=gini, n_estimators=10;, score=0.914 total time=   1.6s
[CV 5/5] END ...criterion=gini, n_estimators=10;, score=0.909 total time=   1.6s
[CV 1/5] END ...criterion=gini, n_estimators=50;, score=0.936 total time=   8.7s
[CV 2/5] END ...criterion=gini, n_estimators=50;, score=0.929 total time=   8.5s
[CV 3/5] END ...criterion=gini, n_estimators=50;, score=0.933 total time=   8.2s
[CV 4/5] END ...criterion=gini, n_estimators=50;, score=0.941 total time=   8.6s
[CV 5/5] END ...criterion=gini, n_estimators=50;, score=0.939 total time=   8.0s
[CV 1/5] END ..criterion=gini, n_estimators=100;, score=0.943 total time=  16.5s
[CV 2/5] END ..criterion=gini, n_estimators=100;,

GridSearchCV(estimator=RandomForestClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'n_estimators': [10, 50, 100]},
             verbose=3)

In [ ]:
open_multi_pred_y = tree_grid.predict(open_multi_dataset.x_test)
accuracy_score(open_multi_dataset.y_test, open_multi_pred_y)

0.9372413793103448

In [ ]:
confusion_matrix(open_multi_dataset.y_test, open_multi_pred_y)

array([[2044,    0,    0, ...,    0,    0,    0],
       [   0,   30,    0, ...,    0,    0,    0],
       [   0,    0,   36, ...,    0,    0,    0],
       ...,
       [   0,    0,    0, ...,   32,    0,    0],
       [   0,    0,    0, ...,    0,   32,    0],
       [   0,    2,    0, ...,    0,    0,   41]])

### 모델 저장

- 학습 완료하면 모델 백업 해주세요!

공유 폴더 바로가기를 [내 드라이브] 밑에 생성해두면 바로 연동 됩니다!

In [ ]:
MODEL_ARCHIEVE_PATH = "/content/drive/MyDrive/2023-2ML/models/"

In [ ]:
import joblib
joblib.dump(tree_grid, MODEL_ARCHIEVE_PATH+'open-multi-1.pkl')

['/content/drive/MyDrive/2023-2ML/models/open-multi-1.pkl']

### 모델 불러오기

In [ ]:
model = joblib.load(MODEL_ARCHIEVE_PATH+"open-multi-1.pkl")
model.predict(open_multi_dataset.x_train)

array([ 6, 28, 48, ..., 49, 53, 13])